In [ ]:
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.tools.json.tool import JsonSpec
from langchain.agents import create_json_agent
from langchain_community.utilities.gitlab import GitLabAPIWrapper
from langchain.callbacks import get_openai_callback
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import Tool
from langchain_experimental.utilities import PythonREPL
from langchain import hub
from typing import Optional, Type
from langchain.callbacks.manager import AsyncCallbackManagerForToolRun, CallbackManagerForToolRun
from typing import List
from langchain_community.agent_toolkits.base import BaseToolkit
import dateparser
import operator
import datetime
import json
import re


In [ ]:
llm = ChatOpenAI(model="gpt-4-0613", temperature=0)
data = json.load(open('metadata.json'))
data1 = json.load(open('metadata1.json'))

In [ ]:
relevant_keys = ['title', 'date', 'source', 'type', 'register_num']

def format_string(s):
    relevant_keys = ['title', 'date', 'source', 'type', 'register_num']
    for relevant_key in relevant_keys:
        s = s.replace(relevant_key, f"e['{relevant_key}']")

    types = ['lecture', 'article', 'patent']
    for type_ in types:
        s = s.replace(type_, f"'{type_}'")

    s = s.replace("e['date']", "datetime.datetime.strptime(e[\'date\'], \'%Y-%m-%d\')")

    def replace_four_digit_numbers(text):
        pattern = r"\b(\d{4})\b"
        return re.sub(pattern, r"dateparser.parse('\1')", text)

    return replace_four_digit_numbers(s)

class TeslaToolGetDocuments(BaseTool):
    name = "tesla_tool_get_documents"
    description = """
    Can be used to list all relevant Nikola Tesla document entries from a dictionary in memory based on a condition.
    All entries are about documents written by Nikola Tesla, so don't consider him a source, sources are only considered for article publishers.
    Fields that are available for search are title(str), date(str), source(str), type(str, possible values lecture, article, patent), register_num(str)
    Operations that are available for search are "or, and, ==, <=, >=, <, >, !="
    The tool takes as input a string representing conditions for search delimited with a single whitespace.
    Example tool input:
        type == lecture and date < 1905 and date > 1900
    """
    return_direct: bool = True

    data: List
    
    def _run(self, tool_input: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        parsed_condition = format_string(tool_input)
        return [e for e in self.data if eval(parsed_condition)]

    async def _arun(
        self, tool_input: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        self._run(tool_input)

In [ ]:
custom_tool = TeslaToolGetDocuments(data=data)
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")
tools = [custom_tool]
prompt = hub.pull("hwchase17/react")

In [ ]:
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
res = agent_executor.invoke({"input": "can you give me the list of articles and patents of Nikola Tesla between 1901 and 1905?"})

In [ ]:
agent_executor.tools[0].data

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.prompt_selector import ConditionalPromptSelector


In [ ]:
# from langchain.llms import OpenAI
# from langchain.chains.prompt_selector import ConditionalPromptSelector
# from langchain.prompts import PromptTemplate

# # Set up the four LLMs
# initial_llm = OpenAI(model_name="text-ada-001")
# llm_1 = OpenAI(model_name="text-curie-001")
# llm_2 = OpenAI(model_name="text-babbage-001")
# llm_3 = OpenAI(model_name="text-davinci-002")

# # Define prompts for the three subsequent LLMs
# prompt_1 = PromptTemplate(input_variables=["input"], template="Prompt 1: {input}")
# prompt_2 = PromptTemplate(input_variables=["input"], template="Prompt 2: {input}") 
# prompt_3 = PromptTemplate(input_variables=["input"], template="Prompt 3: {input}")

# # Set up conditional logic to select prompt based on initial LLM output
# prompt_selector = ConditionalPromptSelector(
#     default_prompt=prompt_1,
#     conditionals=[
#         (lambda output: "condition_1" in output, prompt_2),
#         (lambda output: "condition_2" in output, prompt_3)
#     ]
# )

# # Take user input and generate initial LLM output
# user_input = "Enter your prompt here"
# initial_output = initial_llm(user_input)

# # Select prompt based on initial output and generate final output
# selected_prompt = prompt_selector.get_prompt(initial_output)
# final_output = selected_prompt.format_prompt(initial_output).to_string()

In [ ]:
import pandas as pd
import sqlite3

# Load the JSON file into a pandas DataFrame
df = pd.read_json('metadata.json')

# Create a connection to the SQLite database
conn = sqlite3.connect('metadata.db')

# Create a table in the database
df.to_sql('metadata', conn)

# Close the connection
conn.close()

In [ ]:
conn = sqlite3.connect('metadata.db')


In [ ]:
# Execute a SQL query to select all rows from the 'metadata' table
query = "SELECT * FROM metadata"

# Create a cursor object from the connection
cursor = conn.cursor()

# Execute the query and fetch all rows
cursor.execute(query)
result = cursor.fetchall()

# Convert the result to a pandas DataFrame for easier viewing
df_result = pd.DataFrame(result, columns=[column[0] for column in cursor.description])

# Display the DataFrame
print(df_result)


In [ ]:
column_names = df_result.columns
print(column_names)


In [ ]:
df_result[df_result['type'] == 'lecture']

In [ ]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('metadata.db')

# Close the connection
conn.close()
# Create a connection to the SQLite database
conn = sqlite3.connect('metadata.db')

# Execute a SQL query to select all rows from the table
query = "SELECT * FROM metadata"

# Create a cursor object from the connection
cursor = conn.cursor()

# Execute the query and fetch all rows
cursor.execute(query)
result = cursor.fetchall()

# Close the connection
conn.close()

# Convert the result to a pandas DataFrame for easier viewing
df_result = pd.DataFrame(result, columns=[column[0] for column in cursor.description])

# Display the DataFrame
# Load the JSON file
with open('assistant_summary.json') as f:
    json_data = json.load(f)

# Create a DataFrame from the JSON data
df_json = pd.DataFrame(json_data)

# Merge the two DataFrames based on the matching columns
df_merged = pd.merge(df_result, df_json, left_on='assistant_OAI_id', right_on='assistant_id')

# Display the merged DataFrame
print(df_merged)
# print(df_result)

# Create a connection to the SQLite database
conn = sqlite3.connect('metadata.db')

# Save the DataFrame as a table in the database
df_merged.to_sql('metadata', conn, if_exists='replace')

# Close the connection
conn.close()



In [ ]:
titles = df_merged['title'].tolist()
summaries = df_merged['summary'].tolist()
print(titles[10])
print(summaries[10])


In [ ]:
import sqlite3
import pandas as pd

# Create a connection to the SQLite database
conn = sqlite3.connect('metadata.db')

# Execute a SQL query to select all rows from the 'metadata' table
query = "SELECT * FROM metadata"

# Read the query result into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Close the connection
conn.close()

# Display the DataFrame
print(df)


In [ ]:
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI
# from langchain_core.prompts.chat import ChatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


db = SQLDatabase.from_uri("sqlite:///metadata.db")
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
                """
                You have a single table called metadata that contains entries of files by Nikola Tesla, columns of interest are: 'index', 'id', 'title', 'date', 'type', 'source', 'register_num', 'summary'.
                Nikola Tesla is the author of all entries, so he should not be used in queries, or to parse titles.
                Source is available only for articles, and is the publisher of the article.
                Register_num is available only for patents.
                Type can be 'lecture', 'article', or 'patent'.
                """
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, top_k=200, prompt=prompt_template)
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [ ]:
type(agent_executor)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "how many patent was issued by Nikola Tesla?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [ ]:
agent_with_chat_history.invoke(
    {"input": "can you list some of them?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

In [1]:
from langchain_openai import ChatOpenAI
import os
gpt_3_5 = "gpt-3.5-turbo"
gpt_4 = "gpt-4-turbo-preview"

llm3 = ChatOpenAI(model=gpt_3_5, temperature=0)
llm4 = ChatOpenAI(model=gpt_4, temperature=0)
llm = ChatOpenAI(model=gpt_3_5, temperature=0)

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "Multi-agent Collaboration"

In [2]:
# sql agent

from langchain.agents.agent import AgentExecutor, RunnableMultiActionAgent
from langchain.agents import create_openai_tools_agent

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit


db = SQLDatabase.from_uri("sqlite:///metadata.db")
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
                """
                You have a single table called metadata that contains entries of files by Nikola Tesla, columns of interest are: 'index', 'id', 'title', 'date', 'type', 'source', 'register_num', 'summary'.
                Nikola Tesla is the author of all entries, so he should not be used in queries, or to parse titles.
                Source is available only for articles, and is the publisher of the article.
                Register_num is available only for patents.
                Type can be 'lecture', 'article', or 'patent'.
                """
        ),
        MessagesPlaceholder(variable_name="messages"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
# sql_agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True, top_k=200, prompt=prompt_template)
tools = tools = SQLDatabaseToolkit(llm=llm, db=db).get_tools()
agent = RunnableMultiActionAgent(
    runnable=create_openai_tools_agent(llm, tools, prompt_template),
    input_keys_arg=["messages"],
    return_keys_arg=["output"],
)
sql_agent_executor = AgentExecutor(
        name="SQL Agent Executor",
        agent=agent,
        tools=tools,
        callback_manager=None,
        verbose=True,
        max_iterations=15,
        max_execution_time=None,
        early_stopping_method="force",
    )

In [3]:
# llm "agent"

from langchain_core.output_parsers import BaseOutputParser
from langchain.chains import LLMChain

class CustomOutputParser(BaseOutputParser):
    def parse(self, output):
        return {'output': output}

output_parser = CustomOutputParser()
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
                """
                You are an AI assistant.
                """
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

simple_llm_chain = LLMChain(
    llm=llm3,
    prompt=prompt_template,
    output_parser=output_parser
)

In [ ]:
# # llm "agent"

# from langchain_core.output_parsers import BaseOutputParser
# from langchain.chains import LLMChain

# class CustomOutputParser(BaseOutputParser):
#     def parse(self, output):
#         return {'output': output}

# # Create the custom output parser
# output_parser = CustomOutputParser()

# # Modify the LLMChain initialization to include the output parser
# simple_llm_chain = LLMChain(
#     llm=llm3,
#     prompt=prompt_template,
#     output_parser=output_parser
# )

# # simple_llm_chain = LLMChain(
# #     llm=llm,
# #     prompt=prompt_template,
# #     output_parser=output_parser
# # )

In [4]:
# supervisor agent

from langchain.output_parsers.openai_functions import JsonOutputFunctionsParser

members = ["SQL", "CHAT"]
supervisor_system_prompt = (
    "You are a supervisor tasked with managing a conversation between the"
    " following workers: {members}. SQL has an acces to a database contianing metadata of Nikola Tesla's works so it can answer questions like how many articles did Nikola Tesla write etc... but it does not have access to his works so it cannot answer questions about specific documents. "
    " CHAT has access to a GPT-4 model, it should answer questions that do not need any data." 
    " Given the following user request,"
    " respond with the worker to act next. Each worker will perform a"
    " task and respond with their results and status. When finished,"
    " respond with FINISH."
)

options = ["FINISH"] + members

function_def = {
    "name": "route",
    "description": "Select the next role.",
    "parameters": {
        "title": "routeSchema",
        "type": "object",
        "properties": {
            "next": {
                "title": "Next",
                "anyOf": [
                    {"enum": options},
                ],
            }
        },
        "required": ["next"],
    },
}

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", supervisor_system_prompt),
        MessagesPlaceholder(variable_name="messages"),
        (
            "system",
            "Given the conversation above, who should act next?"
            " SQL should be selected if the question is about metadata of Nikola Tesla's works, CHAT should be selected if the question does not need any data. Otherwise, or if there is a loop in the conversation, feel free to select FINISH."
            " Or should we FINISH? Select one of: {options}",
        ),
    ]
).partial(options=str(options), members=", ".join(members))

supervisor_chain = (
    prompt
    | llm.bind_functions(functions=[function_def], function_call="route")
    | JsonOutputFunctionsParser()
)

In [5]:
import operator
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage
import functools

from langgraph.graph import StateGraph, END

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str

def agent_node(state, agent, name):
    result = agent.invoke(state)
    print(name, result)
    if 'text' in result:
        return {"messages": [HumanMessage(content=result['text']["output"], name=name)]}
    return {"messages": [HumanMessage(content=result["output"], name=name)]}


sql_node = functools.partial(agent_node, agent=sql_agent_executor, name="SQL")
chat_node = functools.partial(agent_node, agent=simple_llm_chain, name="CHAT")

workflow = StateGraph(AgentState)
workflow.add_node("SQL", sql_node)
workflow.add_node("CHAT", chat_node)
workflow.add_node("supervisor", supervisor_chain)

for member in members:
    workflow.add_edge(member, "supervisor")

conditional_map = {k: k for k in members}
conditional_map["FINISH"] = END
workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()

In [6]:
o = graph.invoke({"messages": [HumanMessage(content="hello i am jaksa")]})
o['messages'][-1].content

ValueError: Missing some input keys: {'agent_scratchpad'}, ['agent_scratchpad', 'messages'], {'messages': [HumanMessage(content='hello i am jaksa')], 'next': 'CHAT'}

In [36]:
o['messages'][-1].content

'I am an AI assistant here to help and provide information to you. How can I assist you today?'